<a href="https://colab.research.google.com/github/JawadSk12/ML-Lab/blob/main/ML%20LAB%20COMPETITION%205.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings("ignore")


In [10]:
train_df = pd.read_csv("/content/train.csv")
test_df  = pd.read_csv("/content/test.csv")

train_df.head()

,id,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium,Heart Disease
0,0,58,1,4,152,239,0,0,158,1,3.6,2,2,7,Presence
1,1,52,1,1,125,325,0,2,171,0,0.0,1,0,3,Absence
2,2,56,0,2,160,188,0,2,151,0,0.0,1,0,3,Absence
3,3,44,0,3,134,229,0,2,150,0,1.0,2,0,3,Absence
4,4,58,1,4,140,234,0,2,125,1,3.8,2,3,3,Presence


In [11]:
print(train_df.shape)
print(test_df.shape)

train_df.info()


(630000, 15)
(270000, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 630000 entries, 0 to 629999
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       630000 non-null  int64  
 1   Age                      630000 non-null  int64  
 2   Sex                      630000 non-null  int64  
 3   Chest pain type          630000 non-null  int64  
 4   BP                       630000 non-null  int64  
 5   Cholesterol              630000 non-null  int64  
 6   FBS over 120             630000 non-null  int64  
 7   EKG results              630000 non-null  int64  
 8   Max HR                   630000 non-null  int64  
 9   Exercise angina          630000 non-null  int64  
 10  ST depression            630000 non-null  float64
 11  Slope of ST              630000 non-null  int64  
 12  Number of vessels fluro  630000 non-null  int64  
 13  Thallium                 630000 n

In [13]:
ID_COL = "id"
TARGET = "Heart Disease"

X = train_df.drop([ID_COL, TARGET], axis=1)
y = train_df[TARGET]

X_test = test_df.drop(ID_COL, axis=1)

print("Train features shape:", X.shape)
print("Test features shape:", X_test.shape)

Train features shape: (630000, 13)
Test features shape: (270000, 13)


In [14]:
X = X.fillna(X.median())
X_test = X_test.fillna(X_test.median())


In [15]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
import numpy as np

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

oof_preds = np.zeros(len(X))
test_preds = np.zeros(len(X_test))


In [16]:
from lightgbm import LGBMClassifier

for fold, (train_idx, val_idx) in enumerate(folds.split(X, y)):
    print(f"Fold {fold + 1}")

    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model = LGBMClassifier(
        n_estimators=500,
        learning_rate=0.05,
        random_state=42
    )

    model.fit(X_train, y_train)

    val_preds = model.predict_proba(X_val)[:, 1]
    oof_preds[val_idx] = val_preds

    print("Log Loss:", log_loss(y_val, val_preds))

    test_preds += model.predict_proba(X_test)[:, 1] / folds.n_splits


Fold 1
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 225963, number of negative: 278037
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.065694 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 422
[LightGBM] [Info] Number of data points in the train set: 504000, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.448339 -> initscore=-0.207383
[LightGBM] [Info] Start training from score -0.207383
Log Loss: 0.26768031327401
Fold 2
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 225963, number of negative: 278037
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.039061 seconds.
You can set `force_row_wise=true` to remove the overhead.
And 

In [17]:
print("Overall CV Log Loss:", log_loss(y, oof_preds))



Overall CV Log Loss: 0.26872815970278446


In [18]:
submission = pd.DataFrame({
    "id": test_df["id"],
    "target": test_preds
})

submission.head()


,id,target
0,630000,0.954393
1,630001,0.009442
2,630002,0.985515
3,630003,0.006626
4,630004,0.174156


In [19]:
submission.to_csv("submission.csv", index=False)
print("submission.csv saved successfully!")


submission.csv saved successfully!
